# Задача 3

## Постановка

Выборка представляет собой статистику бросков для различных игроков NBA. Для каждого игрока известны:

    количество успешных бросков в домашних играх (score_home)
    количество бросков в домашних играх (atm_home)
    количество успешных бросков в гостевых играх (score_away)
    количество бросков в гостевых играх (atm_away)

Требуется определить, есть ли разница в успехе бросков у игроков в домашних и гостевых играх.

In [50]:
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as st
import pandas as pd

from arch.bootstrap import IIDBootstrap

Считывание данных

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad-19/master/labs/lab1/lab1_data/nba.csv', header = 0)

In [3]:
df.head()

,Unnamed: 0,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
0,7,A. Johnson,ATL,30,34,24,28,64,52
1,10,A. Johnson,TOR,63,57,44,38,120,82
2,15,Abdur-Rahim,SAC,114,124,84,90,238,174
3,22,Afflalo,DEN,236,284,189,227,520,416
4,23,Afflalo,DET,75,88,62,67,163,129


Преобразование данных к удобному виду

In [25]:
x = df.values

In [26]:
y = np.transpose(np.vstack([x[:,5]/x[:,3], x[:,6]/x[:,4]]))

In [28]:
y[:10]

array([[0.8, 0.8235294117647058],
       [0.6984126984126984, 0.6666666666666666],
       [0.7368421052631579, 0.7258064516129032],
       [0.8008474576271186, 0.7992957746478874],
       [0.8266666666666667, 0.7613636363636364],
       [0.7541713014460512, 0.7957446808510639],
       [0.6666666666666666, 0.723404255319149],
       [0.8577235772357723, 0.8649789029535865],
       [0.7452830188679245, 0.7849462365591398],
       [0.9072847682119205, 0.8974358974358975]], dtype=object)

Наша задача: проверить нулевую гипотезу, что у игроков доля удачных бросков в домашних играх равна доле удачных бросков в гостевых играх, то есть нет выраженного отклонения вбольшую и ли меньшую сторону. 
В наших терминах - среднее по первому стобцу в матрице Y равно среднему по второму столбцу.

Биномиальный критерий подошёл бы в этой задаче, если б количество бросков было одинаково в гостях и дома

## Перестановочный критерий: 

In [37]:
from permute.core import two_sample #двувыборочный

In [57]:
two_sample(y[:, 0], y[:, 1],  alternative='two-sided')

(0.49916000000000005, -0.0033632040097979488)

p-value = 0.5

In [58]:
two_sample(y[:, 0], y[:, 0],  alternative='two-sided')#что ,было бы для идеальных данных

(0.99986, 0.0)

In [41]:
from permute.core import one_sample #одновыборочный, тот же механизм что и в двухвыборочном

In [61]:
one_sample(y[:,0]/(y[:,0]+y[:,1])-0.5, alternative='two-sided')

(0.3573599999999999, -0.0008697742705979659)

In [62]:
one_sample(10*y[:,0]/(10*y[:,0]+y[:,1])-0.5, alternative='two-sided')#что для плохих данных

(0.0, 0.4083547678375434)

Видим, что p-value всегда больше 0,05, так что нулевую гипотезу можно считать верной

Можем посчитать доверительные интервалы, используя бутстреп

In [52]:
bs = IIDBootstrap(y[:, 0] - y[:, 1])
bs.conf_int(np.mean, 100000, method='basic')

array([[-0.0082957 ],
       [ 0.00159739]])

Видно, что ноль в доверительный интервал попадает

## Вывод

При использовании перестановочного критерия p-value получилось больше 0,05, поэтому мы не можем отвергнуть гипотезу о независимости удачности от места игры. Подсчет доверительных интервалов даёт тот же вывод